In [4]:
import grblc.convert as cv

/Users/youngsam/Code/grb_module/grbLC
/Users/youngsam/Code/grb_module/grbLC/examples


In [10]:
# converting a file manually with convert.toFlux() is pretty easy and may give you more
# flexibility in converting as opposed to convert.convertGRB(). all you need is:
#   1. a list or array of magnitudes
#   2. a list or array of magnitude errors, same length as 1.
#   3. the GRB name (for dust extinction calculation)
#   4. photon index and error (for conversion from other bands to R)
# this tutorial will show you how to do this.

In [5]:
# let's look into the magnitude file we want to convert
# the glob module looks for files with the given pattern in a given directory,
# meaning you can automate this process to look for and use all the 
# files you want to convert at once. contact me if you have questions
# about using glob to automate your conversion workflow.
from glob2 import glob

filename = glob("data/*")[-1]
print(filename)

# we can also hardcode the filename... up to you
filename = "data/020405.txt"
print(filename)

data/020405.txt
data/020405.txt


In [6]:
# we'll use pandas to efficiently play around with the data.
import pandas as pd

df = pd.read_csv(filename,
                 sep="\t", # <-- this tells pandas that each value is separated by a tab
                 )
df.head() # take a peak at the data

,Time,Mag,MagErr,Band,GCN
0,63000,>18.5,?,R,1326
1,89554,20.2,0.3,V,1328
2,95974,20.9,0.3,R,1329
3,102454,21.0,0.3,R,1329
4,121066,>19.9,?,R,1333


In [7]:
# looks good, but we should get rid of the limiting magnitudes for conversino
df.drop(df.index[df.MagErr=="?"], inplace=True)

# let's also make sure the data types for each column is correct
# (i.e., float for the magnitudes & magnitude errors). if they aren't
# correct, the program will try to use them as strings (i.e., "0.1" + "0.2"
# instead of 0.1 + 0.2), which isn't what we want.
print(df.dtypes)

Time       int64
Mag       object
MagErr    object
Band      object
GCN        int64
dtype: object


In [8]:
# looks like the magnitude and magnitude error columns need to be cast as floats, since
# an object is not a number.
df.Mag = df.Mag.astype(float)
df.MagErr = df.MagErr.astype(float)
print(df.dtypes)

Time        int64
Mag       float64
MagErr    float64
Band       object
GCN         int64
dtype: object


In [9]:
# we're ready to convert! this is the easy part.
df['Flux'], df['Flux_err'] = cv.toFlux(
    df['Band'],
    df['Mag'],
    df['MagErr'],
    grb="020405", #         <-- this is important for computing the dust extinction to add
    photon_index=2.0, #     <-- I made this value up but you should get this from literature or the grb_attrs.txt file
    photon_index_err=0.1, # <-- I made this value up but you should get this from literature or the grb_attrs.txt file
)

df.head()

,Time,Mag,MagErr,Band,GCN,Flux,Flux_err
1,89554,20.20,0.30,V,1328,1.445652e-13,4.001364e-14
2,95974,20.90,0.30,R,1329,5.597719e-14,1.546707e-14
3,102454,21.00,0.30,R,1329,5.105181e-14,1.410614e-14
5,80944,18.80,0.14,R,1335,3.872676e-13,4.993612e-14
6,182410,21.45,0.10,V,1337,4.571553e-14,4.275384e-15


In [ ]:
# all done! you can either (1) save this dataframe as is to the same file like this:
df.to_csv(filename,
          sep="\t",    # <-- this makes it so each column is separated by a tab
          index=False) # <-- this makes it so the number index from pandas is not included
# NOTE: this will overwrite the file you just loaded

In [11]:
# or (2) modify your dataframe and save it to a new file like this:
new_filename = filename.replace(".txt", "_converted.txt")
df.drop(columns=["Mag", "MagErr"], inplace=True) # get rid of magnitudes
df.insert(1, "Flux", df.pop("Flux"))             # move the flux columns to in front of time
df.insert(2, "Flux_err", df.pop("Flux_err"))     # move the flux columns to in front of time
df.to_csv(new_filename, sep="\t", index=False)
df.head()

,Time,Flux,Flux_err,Band,GCN
1,89554,1.445652e-13,4.001364e-14,V,1328
2,95974,5.597719e-14,1.546707e-14,R,1329
3,102454,5.105181e-14,1.410614e-14,R,1329
5,80944,3.872676e-13,4.993612e-14,R,1335
6,182410,4.571553e-14,4.275384e-15,V,1337
